In [ ]:
import torch
import random
import torch.nn as nn
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
def get_activation_bitflip_hook(bit_positions, flip_ratio, flip_log):
    if isinstance(bit_positions, int):
        bit_positions = [bit_positions]
        
    def hook(module, inp, out):
        with torch.no_grad():
            # out: torch.Tensor
            flat = out.view(-1)
            total = flat.numel()
            num_flips = max(1, int(total * flip_ratio))
            # 랜덤으로 num_flips개 인덱스 선택
            idxs = torch.randperm(total, device=flat.device)[:num_flips]
            for idx in idxs:
                orig = flat[idx].item()
                # float32 비트 패턴→uint32
                i = np.frombuffer(np.float32(orig).tobytes(), dtype=np.uint32)[0]
                # bit flip
                for pos in bit_positions:
                    i ^= (1 << pos)
                # 다시 float32
                flipped = np.frombuffer(np.uint32(i).tobytes(), dtype=np.float32)[0]
                flat[idx] = torch.as_tensor(flipped, dtype=flat.dtype, device=flat.device)
                flip_log.append((idx.item(), orig, flipped, bit_positions.copy()))
        return out
    return hook


In [ ]:
def get_gradient_bitflip_hook(bit_positions, flip_ratio, flip_log):
    if isinstance(bit_positions, int):
        bit_positions = [bit_positions]
        
    def hook(grad):
        with torch.no_grad():
            flat = grad.view(-1)
            total = flat.numel()
            num_flips = max(1, int(total * flip_ratio))
            idxs = torch.randperm(flat.numel(), device=flat.device)[:num_flips]
            for idx in idxs:
                orig = flat[idx].item()
                # float32 → uint32 비트 패턴
                i = np.frombuffer(np.float32(orig).tobytes(), dtype=np.uint32)[0]
                # 지정한 위치 비트 flip
                for pos in bit_positions:
                    i ^= (1 << pos)
                # 다시 float32
                flipped = np.frombuffer(np.uint32(i).tobytes(), dtype=np.float32)[0]
                flat[idx] = torch.as_tensor(flipped, dtype=flat.dtype, device=flat.device)
                flip_log.append((idx.item(), orig, flipped, bit_positions.copy()))
        # 원래 shape으로 복원
        return grad
    return hook


In [ ]:
def get_layer_name(model):
    layer_names = []
    
    for name, module in model.named_modules():
        if any(keyword in name.lower() for keyword in ['conv', 'linear', 'fc']):
            layer_names.append(name)
    
    if not layer_names:
        return None  # 혹시 찾은 레이어가 없으면 None 반환

    return layer_names